In [1]:
import pandas as pd
import numpy as np
import extracted_data_reader
from tsai.all import *

C:\Users\Gabriele\AppData\Roaming\Python\Python310\site-packages\tsai\imports.py:346: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem . (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  return torch.device("cpu")


In [11]:
X = extracted_data_reader.read_data_ex(ex_number=1, frame_number="mean")
Y = extracted_data_reader.read_target_ex(ex_number=1)

229
229


In [15]:
X = np.array(X)
Y = np.array(Y)
(X.shape, Y.shape)
splits = [[i for i in range(len(X) //2)], [i for i in range(len(X) // 2, len(X))]]
# tfms  = [None, [Categorize()]]
# dsets = TSDatasets(X, Y, tfms=tfms, splits=splits, inplace=True)
# dsets

TypeError: unhashable type: 'numpy.ndarray'

In [4]:
Y

array([[ 4,  4,  5,  5,  5, 23],
       [ 4,  4,  5,  5,  5, 23],
       [ 4,  4,  5,  5,  5, 23],
       ...,
       [ 5,  5,  5,  5,  4, 24],
       [ 5,  5,  5,  5,  4, 24],
       [ 5,  5,  5,  5,  4, 24]])